In [ ]:
#!pip install 'continual>=2.0.0a'
#!pip install sagemaker
#!pip install sagemaker-experiments
#!pip install pandas
#!pip install boto3>=1.15.0
#!pip install joblib

In [ ]:
from sagemaker.sklearn import SKLearn
from sagemaker import estimator
import sagemaker
import boto3
import time
import pandas as pd
import os
import yaml

with open('params.yaml') as config_file:
    config = yaml.safe_load(config_file)

In [35]:
# Create SageMaker session
sagemaker_session = sagemaker.Session()

role = config['environment']['sm_role']

s3 = boto3.client('s3')

In [36]:
continual_api_key = 'apikey/c8d36975338347718850b8f6f5a3975f'

In [37]:
# Create estimator
estimator = SKLearn(
    entry_point='run.py',
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='1.0-1',
    py_version='py3',
    source_dir='.', 
    environment={"CONTINUAL_APIKEY": continual_api_key}
)

In [38]:
train_sklearn = "sklearn-training-job-{}".format(int(time.time()))

estimator.fit(
    job_name=train_sklearn,
)

INFO:sagemaker:Creating training-job with name: sklearn-training-job-1674854001


2023-01-27 21:13:23 Starting - Starting the training job...
2023-01-27 21:13:41 Starting - Preparing the instances for training......
2023-01-27 21:14:34 Downloading - Downloading input data...
2023-01-27 21:15:00 Training - Downloading the training image...
2023-01-27 21:15:36 Training - Training image download completed. Training in progress.2023-01-27 21:15:39,310 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-27 21:15:39,314 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:15:39,322 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-27 21:15:39,672 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 11.2 MB/s eta 0:00:00
     ━━━━

UnexpectedStatusException: Error for Training job sklearn-training-job-1674854001: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/training.py", line 31, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/entry_point.py", line 99, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/process.py", line 286, in run
    process = check_error(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/process.py", line 204, in ch

In [ ]:
os.environ['SM_MODEL_DIR']

In [ ]:
# Deployment
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = estimator.transformer(
    instance_count=1,
    instance_type="ml.m5.xlarge",
    assemble_with="Line",
    accept="text/csv"
)

test_data_path = sagemaker_session.upload_data('chimpanzee.txt')

# Feed the test data
transformer.transform(test_data_path, content_type="text/csv")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
output = transformer.output_path
print(output)

# Log endpoint to Continual
#https://github.com/continual-ai/continual/blob/main/integration_test/test_sdk_endpoints.py